<div style="font-size: 3em; font-weight: bold; text-align: center;">Resampling</div>
<div style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</div>

In [ ]:
import numpy as np

# Resampling methods
Generalmente, no existe un modelo estadístico que sea capaz de explicar un fenómeno. Pero si que podemos decir que cuanto más complejo es el modelo, mejor será capaz de *fit*/*encajar* con los datos de entrenamiento. Pero esto no indica que sea bueno para predecir -> Esto es lo que se llama **OVERFITTING**.

También hay que tener en cuenta que cuando tenemos un dataset, es posible que no tengamos suficientes datos, o que existan datos atípicos que no deberían ser relevantes. Para solucionar algunos de estos problemas existe el **resampleo**

Las técnicas de **resampleo** se basan en usar las muestras de nuestro dataset original y volver a generar un nuevo dataset. A partir de este nuevo dataset crearemos un modelo de interés y conseguiremos informacion adicional del modelo creado.

## Training - Validation - Test 
Para entender que son los *resampling methods* debemos saber la diferencia entre el *test error rate* y el *training error rate*.

* El *test error rate* es el error medio que resulta de usar un método de aprendizaje estadístico para predecir la respuesta de nuevas observaciones, estas son las muestras de test. Se desea que este sea siempre lo menor posible, ya que esta prediciendo correctamente puntos nuevos que no han sido usados para entrenar
* El *training error rate* es el error medio que resulta de usar un método de aprendizaje estadístico en las muestras de entrenamiento.

En el caso de no tener las muestras de test suficientes para verificar que nuestro *test error rate* es real, debermos usar algunas de las siguientes técnicas para poder acercarnos mejor a una repsuesta.

## Cross validation
Agrupa un conjunto de métodos de **resampleo** basado en la elección de *sets de entrenamiento* y *sets de training*. El proceso más utilizado es el descrito en [k-fold](#k-fold-cross-validation---validacion-cruzada).

### The validation Set Approach.
Es una tarea sencilla que involucra dividir de manera aleatoria nuestros datos observados en **dos partes**: **set de entrenamiento** y **set de test**. El modelo se crea basándonos en el set de entrenamiento y el error de test se realiza usando las observaciones guardadas en test (generalmente se usa el MSE).

Como la división entre training y test es aleatoria, si realizamos otra división, entonces nos saldrá distintos valores de MSE para cada división.

Dos de las mayores desventajas de este proceso son:
* La estimación del *test error* puede variar bastante entre divisiones.
* Solo se usan una parte de las muestras observadas para entrenar al modelo, la otra parte se usa para test.

### Leave-One-Out Cross-validation LOOCV
Es muy parecida al Validation set aproach, pero intenta resolver sus desventajas.

Se basa en dividir las observaciones en dos partes. Pero en vez de crear dos set de datos con tamaños similares, se crea un <u>set de test con 1 SOLA OBSERVACION y el resto se usa para training.</u>

Usaremos esta única observación predictiva para realizar el MSE, la ventaja es que al haber usado todas las obsevaciones disponibles, el modelo se habrá entrenado con casi todas y será unbiased, el problema es que si solo nos basamos en esta observación , existirá mucha varianza. Así que deberemos realizar este proceso con las demás muestras y sacar la media del MSE.

$$
CV_{n} = \frac{1}{n} \sum_{i=1}^{n} MSE_i
$$

Las ventajas de este procedimiento son:

* Tiene menos sesgo o bias
* Usa la mayoría de las observaciones, con lo cual no sobreestima tanto el error
* No depende de la aleatoriedad

Las desventajas son:
* Son muchos cálculos, por que tienes que hacerlo n veces, con lo cual tarda mucho

### k-fold Cross-Validation -> Validacion cruzada
Este proceso se basa en dividir las obsevaciones en $k$ grupos de aproximadamente el mismo tamaño. El primer grupo será el de validación en la primera iteracción, con este pimer grupo como test y los demás $k-1$ grupos como trainning, calcularemos el error para ese modelo. Repetimos este proceso usando los demás *folds* como test, hasta que todos hayan sido en algún momento *test*.

Se realiza de la siguiente forma:

Desde i=1 hasta k:
    El gupo i es de test
    Construir modelo con el resto de grupos
    Estimar el error
Realizar la media de los errores

$$
CV_{k} = \frac{1}{n} \sum_{i=1}^n Error estimado
$$

In [ ]:
def cv_linear(X, y, k):
    n = len(y)
    m = int(n/k)
    S = 0

    for i in range(k):
        test = list(range(i*k, (i+1)*k))
        training = list(set(range(n))-set(test))

        beta = np.linalg.inv(X[training,].T@X[training,])@X[training,].T@y[training]
        e = y[test] - X[test]@beta

        S = S+np.linalg.norm(e)**2

    return S/n 

Como se puede observar el LOOCV es un caso especial de CV, la mayor ventaja que hay con respecto a lo anterior es el tiempo computacional usado

### Bias variance Trade-off for K-Fold Cross-Validation
Un potencial que tiene la validación cruzada con respecto a LOOCV es que hay veces que puede dar una mejor estimación del error, esto tiene que ver con el trade-off entre bias-variance.

Como hemos visto en los anteriores apartados,

* **SESGO**:Validation set aproach sobreestima la tasa de error , puesto que se usan unas cuantas muestras para training y otras para test, lo cual hace que sea sesgada. En cambio LOOCV al usar la media de la tasa de error medio entre todas las observaciones, entoces será sin sesgo. Para cross validation existirá una un nivel intermedio de sesgo, puesto que une ambos mundos.
* **VARIANZA**: Ahora en la varianza, sabemos que el LOOCV es el que mas varianza tiene porque solo utiliza una observación en el test

### Cross validation en problemas de clasificación.


## BOOTSTRAP
En el mundo real muchas veces trabajamos con poblaciones de datos muy limitadas y coseguir nuevas muestras es muy difícil o muy caro, por ello intentaremso sacar set de datos independientes de la población que tenemos.

**Bootstrap**: Proceso donde obtendremos distintos datasets mediante el **remuestreo constate desde el dataset original**. Los remuestreos se realizan con **REEMPLAZAMIENTO**, lo que significa que una observación puede ser escogida varias veces en el dataset creado por el bootstrap.

Supongamos que para cada fila de nuestro *dataframe* podemos obtener una estimación del parámetro de inferencia $\alpha$ usando una función $f$. 
$$
\hat\alpha = f(df_1, \cdots, df_N)
$$
Si repitieras el proceso de estimar $\alpha$ muchas veces con distintas muestras podríamos tener una aproximación muy buena de lo que sería en realidad $\alpha$. Además al tener *reemplazo*, podemos tener una aproximación menos sesgada.
La varianza $\sigma^2(\hat\alpha)$ será:
$$
\sigma^2(\hat\alpha) := \frac{1}{r-1}\sum^r_{h=1}(\hat\alpha_h-\frac{1}{r}\sum^r_{f=1}\hat\alpha_f)^2
$$

En el Bootstrap debemos indicar 2 hiperparametros:
* $B$: número de veces que vamos a crear un Bootstrap dataset.
* $N$: Numero de observaciones para el Bootstrap datset.

